In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

# Standard Normal CDF
normal = tfp.distributions.Normal(loc=0.0, scale=1.0)

def compute_greeks(S, K, T, r, sigma, option_type="call"):
    """Compute Black-Scholes price and Greeks."""
    
    # Ensure all inputs are float64 to prevent dtype mismatches
    S = tf.cast(S, tf.float64)
    K = tf.cast(K, tf.float64)
    T = tf.cast(T, tf.float64)
    r = tf.cast(r, tf.float64)
    sigma = tf.cast(sigma, tf.float64)

    T = tf.maximum(T, 1e-6)  # Prevent division by zero
    d1 = (tf.math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * tf.sqrt(T))
    d2 = d1 - sigma * tf.sqrt(T)

    N_d1 = normal.cdf(d1)
    N_d2 = normal.cdf(d2)
    
    gamma = normal.prob(d1) / (S * sigma * tf.sqrt(T))
    vega = S * normal.prob(d1) * tf.sqrt(T)
    
    if option_type == "call":
        price = S * N_d1 - K * tf.exp(-r * T) * N_d2
        delta = N_d1
        theta = (-S * normal.prob(d1) * sigma / (2 * tf.sqrt(T))) - r * K * tf.exp(-r * T) * N_d2
        rho = K * T * tf.exp(-r * T) * N_d2
    elif option_type == "put":
        price = K * tf.exp(-r * T) * (1 - N_d2) - S * (1 - N_d1)
        delta = N_d1 - 1
        theta = (-S * normal.prob(d1) * sigma / (2 * tf.sqrt(T))) + r * K * tf.exp(-r * T) * (1 - N_d2)
        rho = -K * T * tf.exp(-r * T) * (1 - N_d2)

    return {
        "Price": price.numpy(),
        "Delta": delta.numpy(),
        "Gamma": gamma.numpy(),
        "Vega": vega.numpy(),
        "Theta": theta.numpy(),
        "Rho": rho.numpy()
    }

# Define stock parameters
S = 237.45  # Stock price
r = 0.02    # Risk-free rate
sigma = 0.419  # Implied volatility
T = 0.119  # Time to expiration

# Strike prices for visualization
strike_prices = np.linspace(S * 0.8, S * 1.2, 50)  # Vary K from 80% to 120% of S
time_to_expirations = np.linspace(0.01, 1, 50)  # Vary T from near 0 to 1 year

# Compute Greeks for different strike prices
greeks_by_strike = {g: [] for g in ["Delta", "Gamma", "Vega", "Theta", "Rho"]}
for K in strike_prices:
    greeks = compute_greeks(S, K, T, r, sigma, option_type="call")
    for key in greeks_by_strike:
        greeks_by_strike[key].append(greeks[key])

# Compute Greeks for different time to expirations
greeks_by_time = {g: [] for g in ["Delta", "Gamma", "Vega", "Theta", "Rho"]}
for T_val in time_to_expirations:
    greeks = compute_greeks(S, S, T_val, r, sigma, option_type="call")  # At-the-money
    for key in greeks_by_time:
        greeks_by_time[key].append(greeks[key])

# Plot Greeks vs. Strike Price
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
greek_names = ["Delta", "Gamma", "Vega", "Theta", "Rho"]
for i, key in enumerate(greek_names):
    ax = axes[i // 3, i % 3]
    ax.plot(strike_prices, greeks_by_strike[key], label=key, color="b")
    ax.set_title(f"{key} vs. Strike Price")
    ax.set_xlabel("Strike Price (K)")
    ax.set_ylabel(key)
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()

# Plot Greeks vs. Time to Expiration
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, key in enumerate(greek_names):
    ax = axes[i // 3, i % 3]
    ax.plot(time_to_expirations, greeks_by_time[key], label=key, color="r")
    ax.set_title(f"{key} vs. Time to Expiration")
    ax.set_xlabel("Time to Expiration (Years)")
    ax.set_ylabel(key)
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()

InvalidArgumentError: cannot compute Sub as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Sub] name: 